# Примеры работы с Exoplanet AI

В этом ноутбуке показаны примеры использования библиотеки Exoplanet AI для загрузки, предобработки и анализа данных с целью поиска транзитов экзопланет.

## Содержание

1. Загрузка данных
2. Предобработка
3. Разбиение на окна
4. Визуализация
5. Аугментация данных
6. Сохранение

In [ ]:
# Импортируем необходимые библиотеки
import numpy as np
import matplotlib.pyplot as plt
from src import preprocess, model, detect, visualize

# Настройка стиля графиков
plt.style.use('seaborn-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Загрузка данных

В этом разделе мы загрузим данные для известной звезды с подтвержденной экзопланетой - Kepler-10. Этот объект интересен тем, что у него есть два подтвержденных планетных компаньона:

In [ ]:
# Загрузим данные с кэшированием
times, flux = preprocess.load_lightcurve(
    "Kepler-10",
    mission=preprocess.DataSource.KEPLER,
    use_cache=True
)

# Проверим качество данных
validation = preprocess.validate_lightcurve(times, flux)
print("Информация о качестве данных:")
print(f"Временной интервал: {validation['time_span']:.1f} дней")
print(f"Медианный интервал: {validation['median_cadence']:.3f} дней")
print(f"Количество пропусков: {validation['n_gaps']}")
print(f"Количество выбросов: {validation['n_outliers']}")

# Построим исходные данные
plt.figure(figsize=(15, 5))
plt.plot(times[:1000], flux[:1000], 'k.', alpha=0.5, ms=1)
plt.xlabel('Время [дни]')
plt.ylabel('Нормированный поток')
plt.title('Исходные данные (первые 1000 точек)')
plt.show()

## 2. Предобработка

Теперь проведем полный цикл предобработки данных:
1. Удалим выбросы
2. Удалим тренд
3. Сгладим шумы
4. Нормализуем данные

In [ ]:
# Создадим функцию для визуализации результатов предобработки
def plot_preprocessing_step(times, flux_before, flux_after, title):
    plt.figure(figsize=(15, 8))
    
    plt.subplot(211)
    plt.plot(times[:1000], flux_before[:1000], 'k.', alpha=0.5, ms=1)
    plt.title(f'{title} - До')
    plt.ylabel('Поток')
    
    plt.subplot(212)
    plt.plot(times[:1000], flux_after[:1000], 'k.', alpha=0.5, ms=1)
    plt.title(f'{title} - После')
    plt.xlabel('Время [дни]')
    plt.ylabel('Поток')
    
    plt.tight_layout()
    plt.show()

# 1. Удаление выбросов
flux_clean = preprocess.remove_outliers(flux, method='mad')
plot_preprocessing_step(times, flux, flux_clean, 'Удаление выбросов')

In [ ]:
# 2. Удаление тренда
flux_detrend = preprocess.detrend(flux_clean, times, method='polynomial')
plot_preprocessing_step(times, flux_clean, flux_detrend, 'Удаление тренда')

In [ ]:
# 3. Сглаживание
flux_smooth = preprocess.smooth_lightcurve(flux_detrend, method='savgol')
plot_preprocessing_step(times, flux_detrend, flux_smooth, 'Сглаживание')

In [ ]:
# 4. Нормализация
flux_norm = preprocess.normalize_array(flux_smooth, method='robust')
plot_preprocessing_step(times, flux_smooth, flux_norm, 'Нормализация')

## 3. Разбиение на окна

Теперь разобьем наши данные на окна для обучения. Мы создадим синтетический набор данных с известными транзитами для тренировки модели.

In [ ]:
# Создадим синтетический набор данных
X_train, y_train = preprocess.generate_training_dataset(
    num_samples=5000,
    sequence_length=2000,
    transit_probability=0.3
)

print(f"Размеры обучающего набора: {X_train.shape}")
print(f"Позитивных примеров: {y_train.sum()}")
print(f"Негативных примеров: {(1 - y_train).sum()}")

# Визуализируем несколько примеров
plt.figure(figsize=(15, 10))

for i in range(4):
    plt.subplot(2, 2, i+1)
    plt.plot(X_train[i])
    plt.title(f"{'Транзит' if y_train[i] == 1 else 'Нет транзита'}")
    plt.xlabel('Время')
    plt.ylabel('Поток')
    
plt.tight_layout()
plt.show()

## 4. Обучение модели

Используем сгенерированные данные для обучения модели детекции транзитов.

In [ ]:
# Обучим модель
model, history = detect.train_on_windows(
    X_train,
    y_train,
    model_type='cnn',
    epochs=10,
    batch_size=32,
    lr=0.0005
)

# Визуализируем процесс обучения
visualize.plot_training_history(history)

## 5. Поиск транзитов

Теперь применим обученную модель для поиска транзитов в реальных данных.

In [ ]:
# Поиск транзитов в предобработанных данных
probs = detect.sliding_prediction_full(model, flux_norm)
candidates = detect.extract_candidates(times, probs, threshold=0.7)

print(f"Найдено кандидатов: {len(candidates)}")
for i, candidate in enumerate(candidates):
    print(f"  кандидат {i}: start_time={candidate['start_time']:.3f}, " +
          f"end_time={candidate['end_time']:.3f}, mean_prob={candidate['mean_prob']:.3f}")

# Визуализация результатов
visualize.plot_lightcurve(times, flux_norm, probs, candidates)

# Детальный просмотр первого кандидата
if len(candidates) > 0:
    visualize.plot_candidate_details(times, flux_norm, candidates[0])

## 6. Сохранение результатов

В заключение сохраним предобработанные данные и результаты для последующего использования.

In [ ]:
# Создадим словарь с результатами
results = {
    'times': times,
    'flux_raw': flux,
    'flux_processed': flux_norm,
    'predictions': probs,
    'candidates': candidates
}

# Сохраним в numpy формате
import os
os.makedirs('data/results', exist_ok=True)
np.savez_compressed(
    'data/results/kepler10_results.npz',
    **results
)

print("Результаты сохранены в data/results/kepler10_results.npz")